# Import Dependencies 

In [1]:
from scipy.stats import linregress
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
import requests
import time
import json

## Load the Data

In [2]:
# read the saved data:
music_df = pd.read_csv("../Output_Data/all_data_genre.csv", encoding = "Latin-1")
print(len(music_df))
music_df.head()

817


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,Deezer Album ID,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,Major,80,89,83,31,0,8,4,463574485,Asian Music
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,Major,71,61,74,7,0,10,4,410083687,Rap/Hip Hop
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,Major,51,32,53,17,0,31,6,484372295,Pop
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,Major,55,58,72,11,0,11,15,108447472,Pop
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,Minor,65,23,80,14,63,11,6,442984025,Latin Music


## Remove Duplicated Rows and Unwanted Columns

In [3]:
# Using Pandas remove duplicated Rows
unduplicated_music_df = music_df.drop_duplicates(subset=["track_name", "artist(s)_name",
                                                         "released_year", "released_month", 
                                                         "Deezer Album ID"], keep=False,\
                                                         inplace=False)
len(unduplicated_music_df)

813

In [4]:
# Remove Unwanted Columns:
final_music_df = unduplicated_music_df.drop(columns=["in_shazam_charts", "released_day",
                                                      "mode", "danceability_%", "valence_%",
                                                      "energy_%", "acousticness_%",
                                                      "instrumentalness_%", "liveness_%",
                                                      "speechiness_%", "Deezer Album ID"])
# final_music_df.columns
final_music_df.head(1)

,track_name,artist(s)_name,artist_count,released_year,released_month,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,bpm,key,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,553,147,141381703,43,263,45,10,125,B,Asian Music


### Rename Columns Then Remove Empty Cells

In [5]:
# First, using pandas Rename the columns
final_music_df = final_music_df.rename(columns={"track_name": "Track Name", "artist(s)_name": "Artist(s) Name",
                                                "artist_count": "Artist Count",
                                                "released_year": "Released Year", "released_month": "Released Month", 
                                                "in_spotify_playlists": "Spotify Playlists Ranking",
                                                "in_spotify_charts": "Spotify Charts Ranking", "streams": "Spotify Streams",
                                                "in_apple_playlists": "Apple Playlists Ranking", 
                                                "in_apple_charts": "Apple Charts Ranking",
                                                "in_deezer_playlists": "Deezer Playlists Ranking",
                                                "in_deezer_charts": "Deezer Charts Ranking",
                                                "bpm": "BPM", "key": "Key"})

# Now remove all empty cells - NOTE: most empty cells are in the "Deezer Genre" column
cleaned_final_music_df = final_music_df.dropna()

print(len(cleaned_final_music_df))
cleaned_final_music_df.head(3)

695


,Track Name,Artist(s) Name,Artist Count,Released Year,Released Month,Spotify Playlists Ranking,Spotify Charts Ranking,Spotify Streams,Apple Playlists Ranking,Apple Charts Ranking,Deezer Playlists Ranking,Deezer Charts Ranking,BPM,Key,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,553,147,141381703,43,263,45,10,125,B,Asian Music
1,LALA,Myke Towers,1,2023,3,1474,48,133716286,48,126,58,14,92,C#,Rap/Hip Hop
2,vampire,Olivia Rodrigo,1,2023,6,1397,113,140003974,94,207,91,14,138,F,Pop


In [6]:
# The Track Name ""Love Grows (Where My Rosemary Goes)"" has an error in the 
# "Spotify Streams" Column so we will remove it using drop by index.

# drop by index
cleaned_final_music_df = cleaned_final_music_df.drop([478])
len(cleaned_final_music_df)

694

In [7]:
print(cleaned_final_music_df[["Spotify Streams", "Deezer Playlists Ranking"]].dtypes)

# Convert the values in the "Spotify Streams" AND "Deezer Playlists Ranking" Columns to integer
# First convert the Spotify Streams Column
cleaned_final_music_df["Spotify Streams"] = cleaned_final_music_df["Spotify Streams"].astype("int64")

# For the "Deezer Playlist" column we have to remove the "," then convert the string to an interger.
cleaned_final_music_df["Deezer Playlists Ranking"] = cleaned_final_music_df["Deezer Playlists Ranking"].str.replace(",", "")
cleaned_final_music_df["Deezer Playlists Ranking"] = cleaned_final_music_df["Deezer Playlists Ranking"].astype(int) 

print("-------------------------------------------")

print(cleaned_final_music_df[["Spotify Streams", "Deezer Playlists Ranking"]].dtypes)

Spotify Streams             object
Deezer Playlists Ranking    object
dtype: object
-------------------------------------------
Spotify Streams             int64
Deezer Playlists Ranking    int32
dtype: object


In [8]:
# export to csv:
cleaned_final_music_df.to_csv("../Output_Data/cleaned_song_data.csv", encoding = "Latin-1", index=False, header=True)

<h1><center>Top Songs in 2023 - 2022 - and Prior to 2022</center></h1>

In [9]:
# Define a function that will highlight the background of a specified column.
def highlighting(inp_var, color='lightblue'):
    return [f'background-color: {color}' for i in inp_var]

## Top Songs in 2023: ____________________________________________________________________________

In [10]:
# Songs that were released in 2023:
songs_23_df = cleaned_final_music_df[cleaned_final_music_df["Released Year"] == 2023]

### `Top Songs in 2023 Spotify`

In [11]:
# Top 3 songs in '23 based on Spotify streams

# Sort the "streams" values in DESCENDING order.
top_23_streams_df = songs_23_df.sort_values(by="Spotify Streams", ascending=False) 

# Format the Streams Column for better readability.
top_23_streams_df["Spotify Streams"] = top_23_streams_df["Spotify Streams"].map("{:,}".format)

# Display the Specified Columns and Apply the highlight format:
top_23_streams_df[["Track Name", "Artist(s) Name", "Released Year", \
                  "Released Month", "Spotify Streams", "BPM", "Key", "Deezer Genre"]].\
                  head(3).style.apply(highlighting, axis=0, subset=["Spotify Streams"])


,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Streams,BPM,Key,Deezer Genre
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2023,3,"725,980,112",148,F,Traditional Mexicano
117,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira, Bizarrap",2023,1,"721,975,598",122,D,Latin Music
30,TQG,"Karol G, Shakira",2023,2,"618,990,393",180,E,Rap/Hip Hop


In [12]:
# Top 3 songs in '23 based on Spotify charts

# sort the "in_spotify_charts" in DESCENDING order.
top_23_Spotify_charts = songs_23_df.sort_values(by="Spotify Charts Ranking", ascending=False) 

# Display the Specified Columns and Apply the highlight format:
top_23_Spotify_charts[["Track Name", "Artist(s) Name", "Released Year", \
                  "Released Month", "Spotify Charts Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                  .style.apply(highlighting, axis=0, subset=["Spotify Charts Ranking"])


,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Charts Ranking,BPM,Key,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023,7,147,125,B,Asian Music
2,vampire,Olivia Rodrigo,2023,6,113,138,F,Pop
25,Dance The Night (From Barbie The Album),Dua Lipa,2023,5,101,110,B,Pop


In [13]:
# Top songs in '23 based on Spotify playlists

# sort the "in_spotify_playlists" in DESCENDING order.
top_23_Spotify_playlists = songs_23_df.sort_values(by="Spotify Playlists Ranking", ascending=False) 

# Format the Column for better readability.
top_23_Spotify_playlists["Spotify Playlists Ranking"] = top_23_Spotify_playlists["Spotify Playlists Ranking"]\
                                                                                                .map("{:,}".format)

top_23_Spotify_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                  "Released Month", "Spotify Playlists Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                  .style.apply(highlighting, axis=0, subset=["Spotify Playlists Ranking"])


,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Playlists Ranking,BPM,Key,Deezer Genre
117,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira, Bizarrap",2023,1,"5,724",122,D,Latin Music
50,Boy's a liar Pt. 2,"PinkPantheress, Ice Spice",2023,2,"5,184",133,F,Pop
30,TQG,"Karol G, Shakira",2023,2,"4,284",180,E,Rap/Hip Hop


### `Top Songs in 2023 Apple`

In [14]:
# Top songs in '23 based on Apple charts

# sort the "in_apple_charts" in DESCENDING order.
top_23_Apple_charts = songs_23_df.sort_values(by="Apple Charts Ranking", ascending=False) 

top_23_Apple_charts[["Track Name", "Artist(s) Name", "Released Year", \
                  "Released Month", "Apple Charts Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                  .style.apply(highlighting, axis=0, subset=["Apple Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Charts Ranking,BPM,Key,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2023,7,263,125,B,Asian Music
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2023,3,222,148,F,Traditional Mexicano
5,Sprinter,"Dave, Central Cee",2023,6,213,141,C#,Rap/Hip Hop


In [15]:
# Top songs in 23 based on Apple playlists

# sort the "in_apple_playlists" in DESCENDING order.
top_23_Apple_playlists = songs_23_df.sort_values(by="Apple Playlists Ranking", ascending=False) 

top_23_Apple_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                      "Released Month", "Apple Playlists Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                      .style.apply(highlighting, axis=0, subset=["Apple Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Playlists Ranking,BPM,Key,Deezer Genre
50,Boy's a liar Pt. 2,"PinkPantheress, Ice Spice",2023,2,154,133,F,Pop
44,Baby Don't Hurt Me,"David Guetta, Anne-Marie, Coi Leray",2023,4,145,128,G,Dance
117,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira, Bizarrap",2023,1,119,122,D,Latin Music


### `Top Songs in 2023 Deezer`

In [16]:
# Top songs in 23 based on Deezer charts

# sort the "in_deezer_charts" in DESCENDING order.
top_23_Deezer_charts = songs_23_df.sort_values(by="Deezer Charts Ranking", ascending=False) 

top_23_Deezer_charts[["Track Name", "Artist(s) Name", "Released Year", \
                      "Released Month", "Deezer Charts Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                      .style.apply(highlighting, axis=0, subset=["Deezer Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Charts Ranking,BPM,Key,Deezer Genre
25,Dance The Night (From Barbie The Album),Dua Lipa,2023,5,38,110,B,Pop
117,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira, Bizarrap",2023,1,29,122,D,Latin Music
12,Daylight,David Kushner,2023,4,24,130,D,Pop


In [17]:
# Top songs in 23 based on Deezer playlists

# sort the "in_deezer_playlists" in DESCING order.
top_23_Deezer_playlists = songs_23_df.sort_values(by="Deezer Playlists Ranking", ascending=False)

top_23_Deezer_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                      "Released Month", "Deezer Playlists Ranking", "BPM", "Key", "Deezer Genre"]].head(3)\
                      .style.apply(highlighting, axis=0, subset=["Deezer Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Playlists Ranking,BPM,Key,Deezer Genre
117,"Shakira: Bzrp Music Sessions, Vol. 53","Shakira, Bizarrap",2023,1,254,122,D,Latin Music
44,Baby Don't Hurt Me,"David Guetta, Anne-Marie, Coi Leray",2023,4,213,128,G,Dance
30,TQG,"Karol G, Shakira",2023,2,184,180,E,Rap/Hip Hop


## Top Songs in 2022: ____________________________________________________________________________

In [18]:
# Songs that were released in 2022:
songs_22_df = cleaned_final_music_df[cleaned_final_music_df["Released Year"] == 2022]

### `Top Songs in 2022 Spotify`

In [19]:
# Top songs in '22

# Sort the "streams" values in DESCENDING order.
top_22_streams_df = songs_22_df.sort_values(by="Spotify Streams", ascending=False) 

# Format the Streams Column for better readability.
top_22_streams_df["Spotify Streams"] = top_22_streams_df["Spotify Streams"].map("{:,}".format)

# Display the Specified Columns and Apply the highlight format:
top_22_streams_df[["Track Name", "Artist(s) Name", "Released Year", \
                  "Released Month", "Spotify Streams", "BPM", "Key", "Deezer Genre"]].\
                  head(3).style.apply(highlighting, axis=0, subset=["Spotify Streams"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Streams,BPM,Key,Deezer Genre
73,Me Porto Bonito,"Chencho Corleone, Bad Bunny",2022,5,"1,440,757,818",92,C#,Latin Music
61,"Quevedo: Bzrp Music Sessions, Vol. 52","Bizarrap, Quevedo",2022,7,"1,356,565,093",128,D,Rap/Hip Hop
161,Titi Me Preguntï¿,Bad Bunny,2022,5,"1,264,310,836",107,F,Latin Music


In [20]:
# Top songs in '22 based on Spotify charts

# sort the "in_spotify_charts" in DESCENDING order.
top_22_Spotify_charts = songs_22_df.sort_values(by="Spotify Charts Ranking", ascending=False) 

top_22_Spotify_charts[["Track Name", "Artist(s) Name", "Released Year", \
                      "Released Month", "Spotify Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                      head(3).style.apply(highlighting, axis=0, subset=["Spotify Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Charts Ranking,BPM,Key,Deezer Genre
28,Creepin',"The Weeknd, 21 Savage, Metro Boomin",2022,12,88,98,C#,Rap/Hip Hop
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,80,128,G,Dance
13,Kill Bill,SZA,2022,12,77,89,G#,R&B


In [21]:
# Top songs in '22 based on Spotify playlists

# sort the "in_spotify_playlists" in DESCENDING order.
top_22_Spotify_playlists = songs_22_df.sort_values(by="Spotify Playlists Ranking", ascending=False)

# Format the Column for better readability.
top_22_Spotify_playlists["Spotify Playlists Ranking"] = top_22_Spotify_playlists["Spotify Playlists Ranking"]\
                                                                                                .map("{:,}".format)

top_22_Spotify_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                          "Released Month", "Spotify Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                          head(3).style.apply(highlighting, axis=0, subset=["Spotify Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Playlists Ranking,BPM,Key,Deezer Genre
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,"12,482",128,G,Dance
737,BREAK MY SOUL,Beyoncï¿,2022,6,"9,724",115,C#,Pop
29,Anti-Hero,Taylor Swift,2022,10,"9,082",97,E,Pop


### `Top Songs in 2022 Apple`

In [22]:
# Top songs in '22 based on Apple charts

# sort the "in_apple_charts" in DESCENDING order.
top_22_Apple_charts = songs_22_df.sort_values(by="Apple Charts Ranking", ascending=False) 

top_22_Apple_charts[["Track Name", "Artist(s) Name", "Released Year", \
                    "Released Month", "Apple Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                    head(3).style.apply(highlighting, axis=0, subset=["Apple Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Charts Ranking,BPM,Key,Deezer Genre
636,MANIAC,Stray Kids,2022,3,188,120,C#,Pop
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,184,128,G,Dance
13,Kill Bill,SZA,2022,12,162,89,G#,R&B


In [23]:
# Top songs in '22 based on Apple playlists

# sort the "in_apple_playlists" in DESCENDING order.
top_22_Apple_playlists = songs_22_df.sort_values(by="Apple Playlists Ranking", ascending=False) 

top_22_Apple_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                        "Released Month", "Apple Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                        head(3).style.apply(highlighting, axis=0, subset=["Apple Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Playlists Ranking,BPM,Key,Deezer Genre
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,291,128,G,Dance
647,About Damn Time,Lizzo,2022,4,242,109,A#,Pop
29,Anti-Hero,Taylor Swift,2022,10,242,97,E,Pop


### `Top Songs in 2022 Deezer`

In [24]:
# Top songs in '22 based on Deezer charts

# sort the "in_deezer_charts" in DESCENDING order.
top_22_Deezer_charts = songs_22_df.sort_values(by="Deezer Charts Ranking", ascending=False) 

top_22_Deezer_charts[["Track Name", "Artist(s) Name", "Released Year", \
                    "Released Month", "Deezer Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                    head(3).style.apply(highlighting, axis=0, subset=["Deezer Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Charts Ranking,BPM,Key,Deezer Genre
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,45,128,G,Dance
73,Me Porto Bonito,"Chencho Corleone, Bad Bunny",2022,5,26,92,C#,Latin Music
199,CUFF IT,Beyoncï¿,2022,7,26,115,G,Pop


In [25]:
# Top songs in '22 based on Deezer playlists

# sort the "in_deezer_playlists" in DESCENDING order.
top_22_Deezer_playlists = songs_22_df.sort_values(by="Deezer Playlists Ranking", ascending=False) 

top_22_Deezer_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                        "Released Month", "Deezer Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                        head(3).style.apply(highlighting, axis=0, subset=["Deezer Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Playlists Ranking,BPM,Key,Deezer Genre
37,I'm Good (Blue),"Bebe Rexha, David Guetta",2022,8,537,128,G,Dance
589,Bam Bam (feat. Ed Sheeran),"Camila Cabello, Ed Sheeran",2022,3,492,95,G#,Pop
243,DESPECHï¿,ROSALï¿½,2022,7,422,130,G,Pop


## Top Songs Prior 2022: ____________________________________________________________________________

In [26]:
# all songs before 2022:
music_prior_df = cleaned_final_music_df[cleaned_final_music_df["Released Year"] < 2022]

### `Top Songs Prior 2022 Spotify`

In [27]:
# Sort the "streams" values in DESCENDING order.
top_prior_streams = music_prior_df.sort_values(by="Spotify Streams", ascending=False) 

# Format the Column for better readability.
top_prior_streams["Spotify Streams"] = top_prior_streams["Spotify Streams"].map("{:,}".format)
                                                                                                
# Display the Specified Columns and Apply the highlight format:
top_prior_streams[["Track Name", "Artist(s) Name", "Released Year", \
                "Released Month", "Spotify Streams", "BPM", "Key", "Deezer Genre"]].\
                head(3).style.apply(highlighting, axis=0, subset=["Spotify Streams"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Streams,BPM,Key,Deezer Genre
151,Shape of You,Ed Sheeran,2017,1,"3,562,543,890",96,C#,Pop
36,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2018,10,"2,808,096,550",90,D,Rap/Hip Hop
137,One Dance,"Drake, WizKid, Kyla",2016,4,"2,713,922,350",104,C#,Rap/Hip Hop


In [28]:
# Top songs PRIOR '22 based on Spotify charts

# sort the "in_spotify_charts" in DESCENDING order.
top_prior_Spotify_charts = music_prior_df.sort_values(by="Spotify Charts Ranking", ascending=False)

top_prior_Spotify_charts[["Track Name", "Artist(s) Name", "Released Year", \
                        "Released Month", "Spotify Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                        head(3).style.apply(highlighting, axis=0, subset=["Spotify Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Charts Ranking,BPM,Key,Deezer Genre
3,Cruel Summer,Taylor Swift,2019,8,100,170,A,Pop
41,Starboy,"The Weeknd, Daft Punk",2016,9,79,186,G,R&B
36,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2018,10,78,90,D,Rap/Hip Hop


In [29]:
# Top songs PRIOR '22 based on Spotify playlists

# sort the "in_spotify_playlists" in DESCENDING order.
top_prior_Spotify_playlists = music_prior_df.sort_values(by="Spotify Playlists Ranking", ascending=False) 

# Format the Column for better readability.
top_prior_Spotify_playlists["Spotify Playlists Ranking"] = top_prior_Spotify_playlists\
                                                        ["Spotify Playlists Ranking"].map("{:,}".format)

top_prior_Spotify_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                            "Released Month", "Spotify Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                            head(3).style.apply(highlighting, axis=0, subset=["Spotify Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Spotify Playlists Ranking,BPM,Key,Deezer Genre
640,Get Lucky - Radio Edit,"Pharrell Williams, Nile Rodgers, Daft Punk",2013,1,"52,898",116,F#,Electro
525,Mr. Brightside,The Killers,2003,9,"51,979",148,C#,Rock
606,Wake Me Up - Radio Edit,Avicii,2013,1,"50,887",124,D,Electro


### `Top Songs Prior 2022 Apple`

In [30]:
# Top songs PRIOR '22 based on Apple charts

# sort the "in_apple_charts" in DESCENDING order.
top_prior_Apple_charts = music_prior_df.sort_values(by="Apple Charts Ranking", ascending=False) 

top_prior_Apple_charts[["Track Name", "Artist(s) Name", "Released Year", \
                        "Released Month", "Apple Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                        head(3).style.apply(highlighting, axis=0, subset=["Apple Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Charts Ranking,BPM,Key,Deezer Genre
759,Mary On A Cross,Ghost,2019,9,266,130,B,Metal
3,Cruel Summer,Taylor Swift,2019,8,207,170,A,Pop
736,Master of Puppets (Remastered),Metallica,1986,3,198,105,E,Rock


In [31]:
# Top songs PRIOR '22 based on Apple playlists

# sort the "in_apple_playlists" in DESCENDING order.
top_prior_Apple_playlists = music_prior_df.sort_values(by="Apple Playlists Ranking", ascending=False) 

top_prior_Apple_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                           "Released Month", "Apple Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                           head(3).style.apply(highlighting, axis=0, subset=["Apple Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Apple Playlists Ranking,BPM,Key,Deezer Genre
352,Don't Start Now,Dua Lipa,2019,10,532,124,B,Pop
71,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2021,7,492,170,C#,Pop
137,One Dance,"Drake, WizKid, Kyla",2016,4,433,104,C#,Rap/Hip Hop


### `Top Songs Prior 2022 Deezer`

In [32]:
# Top songs PRIOR '22 based on Deezer charts

# sort the "in_deezer_charts" in DESCENDING order.
top_prior_Deezer_charts = music_prior_df.sort_values(by="Deezer Charts Ranking", ascending=False) 

top_prior_Deezer_charts[["Track Name", "Artist(s) Name", "Released Year", \
                         "Released Month", "Deezer Charts Ranking", "BPM", "Key", "Deezer Genre"]].\
                         head(3).style.apply(highlighting, axis=0, subset=["Deezer Charts Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Charts Ranking,BPM,Key,Deezer Genre
92,Cold Heart - PNAU Remix,"Dua Lipa, Elton John, Pnau",2017,11,37,116,C#,Pop
71,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi",2021,7,31,170,C#,Pop
74,Makeba,Jain,2015,6,22,116,D,Pop


In [33]:
# Top songs PRIOR '22 based on Deezer playlists

# sort the "in_deezer_playlists" in DESCENDING order.
top_prior_Deezer_playlists = music_prior_df.sort_values(by="Deezer Playlists Ranking", ascending=False) 

# Format the Column for better readability.
top_prior_Deezer_playlists["Deezer Playlists Ranking"] = top_prior_Deezer_playlists\
                                                        ["Deezer Playlists Ranking"].map("{:,}".format)

top_prior_Deezer_playlists[["Track Name", "Artist(s) Name", "Released Year", \
                            "Released Month", "Deezer Playlists Ranking", "BPM", "Key", "Deezer Genre"]].\
                            head(3).style.apply(highlighting, axis=0, subset=["Deezer Playlists Ranking"])

,Track Name,Artist(s) Name,Released Year,Released Month,Deezer Playlists Ranking,BPM,Key,Deezer Genre
520,Smells Like Teen Spirit - Remastered 2021,Nirvana,1991,9,"12,367",117,C#,Rock
640,Get Lucky - Radio Edit,"Pharrell Williams, Nile Rodgers, Daft Punk",2013,1,"8,215",116,F#,Electro
777,The Scientist,Coldplay,2002,8,"7,827",146,F,Rock


<h1><center>---------- TOP 3 SONGS IN ALL YEARS ---------- </center></h1>

In [34]:
# --------------------------------------------------------------------------------------------------------
# Sort all values for the columns of interest in DESCENDING order
# SPOTIFY ------------------------------------------------------------------------------------------------
top_streams_df = cleaned_final_music_df.sort_values(by="Spotify Streams", ascending=False) 
top_Spotify_charts = cleaned_final_music_df.sort_values(by="Spotify Charts Ranking", ascending=False) 
top_Spotify_playlists = cleaned_final_music_df.sort_values(by="Spotify Playlists Ranking", ascending=False) 

# APPLE ---------------------------------------------------------------------------------------------------
top_Apple_charts = cleaned_final_music_df.sort_values(by="Apple Charts Ranking", ascending=False) 
top_Apple_playlists = cleaned_final_music_df.sort_values(by="Apple Playlists Ranking", ascending=False)

# DEEZER --------------------------------------------------------------------------------------------------
top_Deezer_charts = cleaned_final_music_df.sort_values(by="Deezer Charts Ranking", ascending=False) 
top_Deezer_playlists = cleaned_final_music_df.sort_values(by="Deezer Playlists Ranking", ascending=False) 
# --------------------------------------------------------------------------------------------------------